In [1]:
from glob import glob
from tqdm import tqdm
#from rdkit import Chem
import numpy as np
import pandas as pd
import re
import shutil
import os

In [2]:
train = pd.read_csv("data/train_set.ReorgE.csv")
train.columns = ["index", "SMILES", "Reorg_g", "Reorg_ex"]

train.head()

,index,SMILES,Reorg_g,Reorg_ex
0,train_0,CC[C@H]1CCCCN1C(=O)[C@@H](C)OC(=O)c1c(C)oc(-n2...,0.631486,0.535060
1,train_1,O[C@@H](CNC1CC1)CN1CCc2sccc2C1,0.825901,1.116781
2,train_2,N#CCCNC(=O)[C@@]1(O)CCSC1,1.463943,0.964848
3,train_3,COC[C@H]1CN(c2ccc(OCC[C@@H](C)O)cc2)C(=O)O1,0.166669,0.161458
4,train_4,N#Cc1c(-c2ccccc2OCC(N)=O)[nH]c(C(N)=O)c1N,0.313820,0.338862


In [ ]:
if os.path.isdir("ex_file"):
    pass
else:
    os.mkdir("ex_file")

if os.path.isdir("g_file"):
    pass
else:
    os.mkdir("g_file")

train_mol = sorted(glob("data/mol_files/train_set/*.mol"))
print(train_mol[0:100])



count=0
for i in train_mol:
    result = []
    tmp = open(i, 'r').read().split("\n")
    #print(tmp)
    #count +=1
    #if count ==100:
    #    break
    print(len(tmp))
    for j in tmp:
        print(count)
        count+=1
        tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')#j.lstrip==> j에서 왼쪽 공백을 지운 문자열  ' +'문자를 ' '로 치환
        #print('check')
        if len(tmp_) > 11:
            result.append(tmp_)
            
    file_name = i.split('/')[-1].split('.')[0]
    
    if "ex" in file_name: #분자 내 원자 위치와 정보를 가지는 csv 파일을 만들어 냄
        pd.DataFrame(result).to_csv(f"ex_file/{file_name}" + ".csv", index = False)
    elif "g" in file_name:
        pd.DataFrame(result).to_csv(f"g_file/{file_name}" + ".csv", index = False)
        

In [4]:
max_len = 0
error = []
g_data, ex_data = [], []
mol = []

for i in tqdm(train[train.columns[0]]):
    tmp_g = pd.read_csv(f"g_file/{i}" + '_g.csv')

    if len(tmp_g) >= max_len:
        max_len = len(tmp_g)
        
    for j in tmp_g["3"]: #어떤 원자인지 C,H,,N,O 등!! 이유는 해당 데이터셋에 등장하는 모든 원소를 추가하기 위함
        mol.append(j) 

100%|██████████| 18157/18157 [00:27<00:00, 652.32it/s]


In [18]:
mol_list = list(set(mol)) #원소의 중복제거=> 즉 등장하는 원소들의 종류가 mol_list가 된다
print(mol_list)

['S', 'C', 'I', 'Br', 'H', 'Si', 'B', 'P', 'Cl', 'F', 'N', 'O']


In [5]:
#mol_list = list(set(mol)) #원소의 중복제거=> 즉 등장하는 원소들의 종류가 mol_list가 된다

g_data, ex_data = [], [] #바닥 상태 데이터와 들뜬 상태 데이터의 리스트 

def get_mol(data):
    if data in mol_list:
        return mol_list.index(data) + 1 #고유 라벨 +1 값을 
    # 'S' : 1, 'C' :2, 'I' : 3, 'Br': 4, 'H': 5, 'Si : 6', 'B' : 7, 'P' : 8 , 'Cl' : 9, 'F' : 10, 'N' : 11, 'O' : 12
    
    else: #해당 원소가 없다면 -1 리턴 ==> 일단 없을리는 없음 , 예외처리
        return -1

for i in tqdm(train[train.columns[0]]):
    tmp_g = pd.read_csv(f"g_file/{i}" + '_g.csv').iloc[:, :4]   #바닥 상태에서 원자에서 0부터 3까지 가져오는데, 해당 원소의 x,y,z 위치와 원소 정보
    tmp_ex = pd.read_csv(f"ex_file/{i}" +'_ex.csv').iloc[:, :4]   #들뜬 상태에서 원자에서 0부터 3까지 가져오는데, 해당 원소의 x,y,z 위치와 원소 정보
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x))  #원소기호를 가져와 해당하는 인덱스로 변환
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))  #원소기호를 가져와 해당하는 인덱스로 변환
    
    tmp_g = np.array(tmp_g)  #읽어온 csv파일의 값을 numpy array 형식으로 변환해줌  
    tmp_ex = np.array(tmp_ex)  #읽어온 csv파일의 값을 numpy array 형식으로 변환해줌  
     
    if max_len != len(tmp_g):  #해당 분자의 원소 수가 전체 데이터 셋중 가장 큰 원소를 가진 개수와 다르다면  빈칸을 0으로 채움 
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 4] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 4] * (max_len - tmp_ex.shape[0]))))
        ?""
        
    elif max_len == len(tmp_g): #해당 길이가 최대길이와 같다면
        tmp_g = tmp_g
        tmp_ex = tmp_ex
    
    g_data.append(tmp_g)
    ex_data.append(tmp_ex)

100%|██████████| 18157/18157 [01:17<00:00, 233.97it/s]


In [17]:
g_data = np.array(g_data).reshape(len(g_data), -1)
ex_data = np.array(ex_data).reshape(len(ex_data), -1)
print(g_data.shape)
print(ex_data.shape)
train_x = np.concatenate((g_data, ex_data), axis = 1)
train_y = np.array(train.loc[:, ["Reorg_g", "Reorg_ex"]])

print(train_x.shape)
print(train_y.shape)


(18157, 1240)
(18157, 1240)
(18157, 2480)
(18157, 2)


In [16]:
train_x

array([[ 2.8972, -2.3256,  0.4309, ...,  0.    ,  0.    ,  0.    ],
       [ 2.2446,  2.0206, -0.4231, ...,  0.    ,  0.    ,  0.    ],
       [ 4.8695, -1.9497, -5.1308, ...,  0.    ,  0.    ,  0.    ],
       ...,
       [ 1.423 , -0.1053,  0.7295, ...,  0.    ,  0.    ,  0.    ],
       [11.486 ,  1.9573,  5.8098, ...,  0.    ,  0.    ,  0.    ],
       [ 0.6476,  0.2797,  0.0464, ...,  0.    ,  0.    ,  0.    ]])

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

In [8]:
LR = MultiOutputRegressor(lgb.LGBMRegressor(random_state=42, n_estimators=1000, max_depth=61, learning_rate=0.075)).fit(train_x, train_y)

In [10]:
if os.path.isdir("test_ex_file"):
    pass
else:
    os.mkdir("test_ex_file")

if os.path.isdir("test_g_file"):
    pass
else:
    os.mkdir("test_g_file")

test_mol = sorted(glob("data/mol_files/test_set/*.mol"))

for i in tqdm(test_mol):
    result = []
    tmp = open(i, 'r').read().split("\n")
    for j in tmp:
        tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
        if len(tmp_) > 11:
            result.append(tmp_)

    file_name = i.split('/')[-1].split('.')[0]

    if "ex" in file_name:
        pd.DataFrame(result).to_csv(f"test_ex_file/{file_name}" + ".csv", index = False)
    elif "g" in file_name:
        pd.DataFrame(result).to_csv(f"test_g_file/{file_name}" + ".csv", index = False)

100%|██████████| 914/914 [00:01<00:00, 519.67it/s]


In [11]:
test = pd.read_csv("data/test_set.csv")

test_g, test_ex = [], []

for i in tqdm(test[test.columns[0]]):
    tmp_g = pd.read_csv(f"test_g_file/{i}" + '_g.csv').iloc[:, :4]
    tmp_ex = pd.read_csv(f"test_ex_file/{i}" +'_ex.csv').iloc[:, :4]
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x))
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))
    
    tmp_g = np.array(tmp_g)
    tmp_ex = np.array(tmp_ex)
    
    if len(tmp_g) < max_len:
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 4] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 4] * (max_len - tmp_ex.shape[0]))))
    elif len(tmp_g) == max_len:
        pass
    else:
        tmp_g = tmp_g[:max_len]
        tmp_ex = tmp_ex[:max_len]
    
    test_g.append(tmp_g)
    test_ex.append(tmp_ex)

100%|██████████| 457/457 [00:01<00:00, 230.71it/s]


In [12]:
test_g = np.array(test_g).reshape(len(test_g), -1)
test_ex = np.array(test_ex).reshape(len(test_ex), -1)

test_x = np.concatenate((test_g, test_ex), axis = 1)

print(test_x.shape)

(457, 2480)


In [13]:
pred = LR.predict(test_x)

In [14]:
submission = pd.read_csv("data/sample_submission.csv")

submission.head()

,index,Reorg_g,Reorg_ex
0,test_0,0,0
1,test_1,0,0
2,test_2,0,0
3,test_3,0,0
4,test_4,0,0


In [15]:
submission.loc[:, ["Reorg_g", "Reorg_ex"]] = pred

submission.to_csv("SAMSUNG_BASELINE.csv", index = False)